In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the data and formating into dataframes

---



## **Showing the provided data**

---

Just using a print to show the summary information of the dataset:

In [ ]:
with open('../input/hashcode-drone-delivery/busy_day.in') as file:
    data_list = file.read().splitlines()

# Printing a summary information of the dataset
print('  ---- General information ---- '
      '\n Rows of the grid: ',data_list[0].split(" ")[0], 
      '\n Columns of the grid: ', data_list[0].split(" ")[1], 
      '\n Number of drones: ', data_list[0].split(" ")[2], 
      '\n Number of turns: ', data_list[0].split(" ")[3],
      '\n Number of the maximun payload in units (u): ', data_list[0].split(" ")[4],
      '\n\n   ---- Product information ----  ',
      '\n Different product types: ',data_list[1],
      '\n Product types weigth: ', data_list[2],
      '\n\n   ---- Warehouses information ----  ',
      '\n Warehouses: ',data_list[3],
      '\n First warehouse location at (row, column): ', data_list[4],
      '\n Inventory of products: ', data_list[5],
      '\n Second warehouse location at (row, column): ', data_list[6],
      '\n Inventory of products: ', data_list[7],
      '\n\n   ---- Orders information ----  ',
      '\n Number of orders: ', data_list[24],
      '\n First order to be delivery at: ', data_list[25],
      '\n Number of items in order: ', data_list[26],
      '\n Items of product types: ', data_list[27])      

## Getting the WareHouse information
---

Reading all the WareHouse information (location) and structuring into a dataframe, with grid **row**, **column**, and **id** for each warehouse:

In [ ]:
# Defining all the warehouses locations
wh_locations = data_list[4:24:2]
# Defining each warehouse row and column location 
wh_id_list = list(range(len(wh_locations)))
wh_rows_list = [ware_house_r.split()[0] for ware_house_r in wh_locations]
wh_cols_list = [ware_house_c.split()[1] for ware_house_c in wh_locations]

# Creating the dataframe with the warehouses locations
warehouse_df = pd.DataFrame({'id': wh_id_list, 'row': wh_rows_list, 'col': wh_cols_list})
warehouse_df = warehouse_df.astype(int)
warehouse_df.head()

## Getting the Products information
---

Get the product information, structuring the amount of each product type for each warehouse:

In [ ]:
# Structuring all the products related with each warehouse
cols = [f'warehouse_{i}' for i in range(len(warehouse_df))]
# Getting all the contents at for each warehouse 
products_df = pd.DataFrame([x.split() for x in data_list[5:24:2]]).T
products_df.columns = cols # Renaming the columns

products_df.head()

In [ ]:
# Including the product weights to each product type
products_df['weight'] = data_list[2].split()

# Including the product ID to each product type
products_df['id'] = list(range(len(products_df)))

# Reordering the dataframe columns
cols = products_df.columns.to_list()
cols.remove("id")
products_df = products_df[["id"] + cols].astype(int)

products_df.head()

In [ ]:
products_df.sum()

## Getting the Orders information
---

In [ ]:
# Get the biggest order in terms of products
max_order = max([len(x.split()) for x in data_list[27:3775:3]])
print(f"The biggest order has {max_order} items!")

# Create the order product number
cols_order = [f'item_{i}' for i in range(max_order)]

# Create the orders dataframe
order_df = pd.DataFrame([x.split() for x in data_list[27:3775:3]]).fillna(0)
order_df.columns = cols_order # Renaming the orders dataframe columns

# Including the order items 
order_df['order_items'] = data_list[26:3775:3]

# Defining the order position in the grid
order_df['coor_x'] = [x.split()[0] for x in data_list[25:3775:3]]
order_df['coor_y'] = [x.split()[1] for x in data_list[25:3775:3]]

# Guaranteing that all columns are numbers
order_df = order_df.astype(int)

order_df.head()

# Analysing the structured data
---

In [ ]:
# Computing the number of products in each warehouse
warehouse_df["prod_count"] = products_df.sum().iloc[1:-1].to_list()

warehouse_df.head(10)

In [ ]:
# Computing the warehouse weight in products
warehouse_weight_list = []
weights = products_df["weight"].to_list()
for col in products_df.columns.to_list()[1:-1]:
    pwh_count = products_df[col].to_list()
    warehouse_weight_list.append(
        sum([q * w for q, w in zip(pwh_count, weights)])
    )

warehouse_df["total_weight"] = warehouse_weight_list

warehouse_df.head(10)

In [ ]:
# Computing the warehouse density
warehouse_df["density"] = warehouse_df["total_weight"] / warehouse_df["prod_count"]

warehouse_df.head(10)

In [ ]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import plotly.express as px

fig = px.scatter(warehouse_df, x="row", y="col", size="density", color="prod_count")
iplot(fig)

# **Building the drone simulator**

---

## Checking each DataFrame

In [ ]:
warehouse_df.head()

In [ ]:
products_df.head()

In [ ]:
order_df.head(20)

## Initializing the data for simulator

In [ ]:
# Get the number of drones
num_drones = int(data_list[0].split(" ")[2])

# Get the number of orders
num_orders = len(order_df)

# Get the number of turns
num_turns = int(data_list[0].split(" ")[3])

# Maximun number deliveries per drone
max_deliveries = int(num_orders // num_drones)

print(f"The number of maximun deliveries per drone: {max_deliveries}")

In [ ]:
num_turns

## Coding the drone simulator class

In [ ]:
import sys
import math
import pickle

class drone:
    """
        The class that simulates the drone movement from warehouse to client and client to warehouse. 
        This drone movement is pretty simple... it receives the orders to deliver and then each drone
        focus on the provided list of deliveries, delivering one item at a time to the client. 
        
        Of course this can be optimized... and it will be further optimized. 
    """
    
    def __init__(self, did=None, delivery_order=None, verbose=False):
        """
            Initializing the drone with some particular content for the simulation. Focus on the 
            `delivery_order` parameter. This parameter carries the information of the orders that 
            this particular drone has to deliver. Notice that the delivery orders are provided as 
            follows: `[ (order_id, [item_1, ..., item_n], [delivery_x, delivery_y]), ... ]`. Which 
            is reestructured in the Start method.
        """
        
        # Drone identification
        self.drone_id = did
        self.verbose = verbose
        
        # Drone status
        self.transporting = False
        self.future_position = [0, 0]
        self.at_order = None
        self.at_product = None
        
        ## Travel information
        # Order informations
        self.delivery_target = None
        
        # Product informations
        self.distance_to_delivery = None
        self.carring_product = None
        
        # The drone internal information
        self._deliveries = delivery_order # [ ("order", "products", "position") ]
        self._delivered_orders = 0
        self._distance = 0
        

    def start(self, wh_logger=None):
        """
            The method responsible to start the drone, and give the first task for each existing 
            drone. Notice that first we will restructure the delivery list into a product list.
            The drone will delivery products to each client location. This function receives the 
            parameter `wh_logger` wich is the logging information of the warehouses and the products
            available at each warehouse. This is needed because the drone will always search for the 
            nearest available product, and then this drone will remove that product from the warehouse
            therefore it will remove the product from the warehouse loggings.
            
            :param tuple wh_logger: A tuple with the products and warehouses dataframe, respectively.
            
            :returns: The modified logger of the products and warehouses.
            :rtype: tuple
        """
        
        # Reordering deliveries
        new_struc_delivery = []
        for delivery in self._deliveries:
            for del_item in delivery[1]:
                new_struc_delivery.append( (delivery[0], del_item, delivery[2]) )
        self._deliveries = new_struc_delivery

        # Create the current product and order
        self.at_order = self._deliveries[0][0]
        self.at_product = self._deliveries[0][1]
        
        if self.verbose:
            self.summary()

        # Compute the distance to the next product
        self.distance_to_delivery, self.future_position, logger = self.find_next_product(wh_logger)

        # Set delivery target to warehouse
        self.transporting = True
        self.delivery_target = "warehouse"
        self.at_product = None

        return logger
    
    
    def step(self, wh_logger=None):
        """
            This function will step the drone one time to the future. If the drone is transporting a product 
            or going to warehouse to get a product, the step method will make the drone move one step closer 
            to its destiny. If the drone is at the client or at the warehouse the step method will deliver the 
            product to the client, or it will get the product from the warehouse, specifically. Then it will 
            guide the drone to the warehouse or the client if the drone is in the client or the warehouse, 
            respectivelly.
            
            :param tuple wh_logger: A tuple with the products and warehouses dataframe, respectively.
            
            :returns: The modified logger of the products and warehouses.
            :rtype: tuple
        """
        
        # ACTION 1 => Move drone if not in target
        if self.transporting:
            # Increase distance 
            self._distance += 1
            # If the position is not the delivery
            if self.distance_to_delivery != 0:
                self.move_step() # Move the drone one step to the target
                return None
            elif self.distance_to_delivery == 0:
                return self.delivery_action(wh_logger) # Load / Unload drone
            else: # If drone distance is None
                print("Drone distance error!")
        
        
    def move_step(self):
        """
            This method is used when the drone is transporting a product.
            
            The move step method is responsible for moving the drone one step closer to its destiny each time 
            the step method asks to move the drone one step closer to its destiny.
        """
        
        if self.drone_id == 2 and self.verbose:
            print(f"Moving drone to {self.distance_to_delivery}")
        
        if self.distance_to_delivery != None:
            self.distance_to_delivery -= 1
    
    
    def delivery_action(self, wh_logger=None):
        """
            This method is used when the drone get to the client or warehouse.
            
            The delivery action method will make the delivery action at the warehouse or at the client, it 
            depends on where the drone is at. 
            
            If the drone is in the client the delivery action will deliver the product to the client, then 
            remove one product from the list of products that the drone must deliver, and then guide the drone
            to the closest warehouse that has the next product that should be delivered. Also it changes the 
            warehouse logging, removing the item that the drone will get from the warehouse logging, making the 
            product "reservation" for this particular drone. Then it computes the tragectory to the warehouse 
            and enter the transport mode to go to the warehouse.
             
            If the drone is in the warehouse, the drone computes the tragectory to the client and get the product
            that it should deliver to the client by setting the `at_product` attribute. At then enter the transport
            mode to go to the client.
            
            :param tuple wh_logger: A tuple with the products and warehouses dataframe, respectively.
            
            :returns: The modified logger of the products and warehouses.
            :rtype: tuple
        """
        
        if self.verbose:
            print(f"Drone {self.drone_id} - At delivery action...")
        
        if self.delivery_target == "warehouse":    
            
            if self.verbose:
                print(f"Drone {self.drone_id} - At the warehouse!")
            
            # Compute the Euclidean distance to the next 
            self.distance_to_delivery = ((self.future_position[0] - self._deliveries[0][2][0])**2 + (self.future_position[1] - self._deliveries[0][2][1])**2)**0.5
            self.distance_to_delivery = math.ceil(self.distance_to_delivery)
            
            # Define the current product carring
            self.at_product = self._deliveries[0][1]
            
            # Define the future position of the drone
            self.future_position = self._deliveries[0][2]
            
            # Set delivery target to client
            self.delivery_target = "client"
            
            if self.verbose:
                print(f"Drone {self.drone_id} - Going to the client...")
            
            return None
            
        elif self.at_product != None and self.delivery_target == "client":
            
            if self.verbose:
                print(f"Drone {self.drone_id} - At the client!")
            
            # Verify if the drone is done
            if len(self._deliveries) != 1: 
                # Remove the current delivery
                self._deliveries = self._deliveries[1:]
                
                # Check if the order is finished
                if self.at_order != self._deliveries[0][0]:
                    self._delivered_orders += 1

                # Create the current product and order
                self.at_order = self._deliveries[0][0]
                self.at_product = self._deliveries[0][1]
                
                if self.verbose:
                    self.summary()
                
                # Compute the distance to the next product
                self.distance_to_delivery, self.future_position, logger = self.find_next_product(wh_logger)
                
                # Set delivery target to warehouse
                self.delivery_target = "warehouse"
                self.at_product = None
                
                return logger
            
            else:
                # Not transporting
                self.transporting = False
                return wh_logger
            
        else: 
            if self.verbose:
                print("Drone delivery action error!")
                
    
    
    def find_next_product(self, wh_logger=None):
        """
            This is the method that is used by delivery action, when at the client, to find the warehouse with 
            the nearest desired product. This method looks at all warehouses logging information to search for 
            the next product that will be delivered, and computes the distance to that warehouse. After it selects
            the closest warehouse for the drone to go and get the product.
            
            :param tuple wh_logger: A tuple with the products and warehouses dataframe, respectively.
            
            :returns: The distance to the next warehouse, the coordinates of the next warehouse, and the modified logger of the products and warehouses.
            :rtype: tuple
        """
        
        # Breaking the logger content
        product_df = wh_logger[0].copy()
        warehouse_df = wh_logger[1].copy()
        
        # Compute the distance to the closest warehouse with the product
        prod_quant = product_df.iloc[self.at_product][1:-1].to_list()
        
        # Initialize the used list
        distance, warehouses = [], []
        for wid, quantity in enumerate(prod_quant):
            # Check product quantity
            if quantity != 0:
                # Include warehouse id
                warehouses.append(wid)
                # Compute warehouse distance
                wh_y, wh_x = warehouse_df.iloc[wid,:][['row', 'col']].tolist()
                dist = ((wh_y - self.future_position[1])**2 + (wh_x - self.future_position[0])**2)**0.5
                # Include warehouse distance
                distance.append(dist)
        
        try:
            # Get the id of the closest warehouse
            minimal_distance = min(distance)
            w_list_id = distance.index(minimal_distance)
            minimal_wh_id = warehouses[w_list_id]

            # Remove the item from the closest warehouse
            product_df.loc[self.at_product, "warehouse_" + str(minimal_wh_id)] -= 1

            # Compute the proper distance
            proper_distance = math.ceil(minimal_distance)

            # Build the logger
            logger = (product_df, warehouse_df)

            return proper_distance, warehouse_df.iloc[w_list_id][["col", "row"]].to_list(), logger
        
        except Exception as e:
            
            error_content = {
                "error": str(e),
                "state": self.write_summary(),
                "product": product_df.to_dict(),
                "warehouse": warehouse_df.to_dict()
            }
            
            with open("./log_error.pickle", "wb") as handle:
                pickle.dump(error_content, handle)
            
            sys.exit("Error!!!!")
        
        
    def write_summary(self):
        """
            Method to create a drone state summary for debug.
        """
        return f"""Drone {self.drone_id} - Going to the warehouse 
                :: order {self.at_order} 
                :: product {self.at_product} 
                :: deliveries {self._delivered_orders}
                :: distance {self._distance}"""
    
    
    def summary(self):
        """
            Method that prints the drone state summary for debug.
        """
        # Print the drone summary
        print(self.write_summary())
    

# Running the simulator

## Creating orders

Creating the initial orders list to delivery for all drones in the format of (order, list of products, delivery coordinates).

In [ ]:
order_list = []
for order in order_df.index.to_list():
    
    # Get the specific order content
    num_items = order_df.iloc[order,-3]
    prod_codes = order_df.iloc[order, :num_items].to_list()
    coordinates = order_df.iloc[order, -2:].to_list()
    
    # Create the order content
    order_content = (order, prod_codes, coordinates)
    
    # Concat the order content
    order_list.append(order_content)

## Creating the logger tuple

Creating the logger to control the information of the warehouse products and the warehouses coordinates that will be provided to the drones.

In [ ]:
def build_logger(products_df, warehouse_df):
    """
        Function that create the logging of the products warehouse disposal and the warehouse location info.
        
        :param pandas.DataFrame products_df: The dataframe with the products warehouse disposal information.
        :param pandas.DataFrame warehouse_df: The dataframe with the warehouse particular informations.
        
        :returns: The logger as the copy of both provided parameters.
        :rtype: tuple
    """
    return (products_df.copy(), warehouse_df.copy()) # Warehouse Products Storage and Warehouse Locations 

## Creating drones

Creating all drones with a specific number of orders to deliver.

In [ ]:
def create_drones(num_drones=None, orders=None, opd=35, verbose=False): 
    """
        Function responsible to create the necessary number of drones to deliver the all the orders provided 
        considering that the mean number of deliveries per drone is provided by `opd`. And that we have a 
        total of `num_drones` to be used.
        
        :param int num_drones: The number of drones.
        :param list orders: The list of orders to be provided to the drones.
        :param int opd: The maen number of orders per drone.
        :param bool verbose: Parameter to show or not debug informations during simulation.
        
        :returns: A list of created drones.
        :rtype: list
    """
    
    # Compute the actual number of drones
    n_drones = math.ceil(len(orders) / opd) 
    
    if n_drones > num_drones:
        n_drones = num_drones
    
    drones = []
    for k in range(n_drones):
        ki = opd * k
        kii = opd * (k + 1)
        if kii > len(orders):
            kii = len(orders) 
        # print(f"Drone with {ki} until {kii} - {kii - ki} items")
        drones.append( drone(did=k+1, delivery_order=orders[ki:kii], verbose=verbose) )
    
    return drones

## Creating the simulator

The simulator is a function that provided the drones, it will simulate all the delivery steps for all the runs.

In [ ]:
def simulate_drones(drones=None, steps=None, logger=None):
    """
        This function will simulate all drones a number of `steps` in to the future, and return the list 
        with all the drones at this particular step in the future.
        
        :param list drones: The list of created drones.
        :param int steps: The number of steps into the future to simulate each drone.
        :param tuple logger: The initial state information of the products in the warehouses and the warehouses particular information.
        
        :retunrs: A list with the simulated drones after the provided number of steps.
        :rtype: list
    """
    
    # Starting the drones
    for drone in drones:
        logger = drone.start(wh_logger=logger)
        
    # Stepping each drone
    for step in range(steps):
        for drone in drones:
            result = drone.step(wh_logger=logger)
            if result != None:
                logger = result
    
    return drones

### Testing the simulator

First we create the logger for the simulation... the "table" with the provided number of products in each warehouse.

Second we create all the drones, based on the orders list that we want to deliver.

Third we simulate the drones `steps` into the future.

In [ ]:
# Building the warehouse and product logger
logger = build_logger(products_df, warehouse_df)

# Creating the drones
drones = create_drones(num_drones=num_drones, orders=order_list, verbose=False)

# Simulating the drones movements to deliveries
sim_drones = simulate_drones(drones=drones, steps=num_turns, logger=logger)

## Creating the cost function

In [ ]:
def algorithm_cost(orders, product_df, warehouse_df, order_df, num_drones, num_orders_per_drone, num_turns):
    """
        Algorithm that based on the list of orders that should be delivered, will simulate all drones and return the 
        percentage of distance that this particular drone moved and the percentage of orders delivered for all the 
        drones.
    """
    
    # Getting the unique orders as integers
    order_ids = list(set([int(i) for i in orders]))
    
    # Computing the order list for all drones
    order_list = []
    for order in order_ids:
        # Get the specific order content
        num_items = order_df.iloc[order,-3]
        prod_codes = order_df.iloc[order, :num_items].to_list()
        coordinates = order_df.iloc[order, -2:].to_list()
        # Concat the order content
        order_list.append( (order, prod_codes, coordinates) )
    
    
    # Building the warehouse and product logger
    logger = build_logger(products_df, warehouse_df)
    
    # Creating the drones for each list created
    drones = create_drones(num_drones=num_drones, opd=num_orders_per_drone, orders=order_list, verbose=False)

    # Simulating the drones movements to deliveries
    sim_drones = simulate_drones(drones=drones, steps=num_turns, logger=logger)  
    
    # Compute the delivery cost
    distance = 0
    delivered_orders = 0
    for drone in sim_drones:
        distance += drone._distance / num_turns
        delivered_orders += drone._delivered_orders / len(orders)

    # Normalizing the distance (percentage)
    distance = 100 * distance / len(sim_drones)
    delivered_orders = 100 * delivered_orders

    # Return the cost for this simulation
    #print(f"-> Testing cost at: {distance + (100 - delivered_orders)}")
    return distance + (100 - delivered_orders) # Mean proportion of distance and proportion of delivered orders

# Running a cost time testing

In [ ]:
import time

num_parameters = len(order_df)
orders_ = list(range(num_parameters))

time_b = time.time() # Save the time before
test_cost = algorithm_cost(orders_, products_df, warehouse_df, order_df, num_drones, 35, 10_000)
time_a = time.time() # Save the time after 

print(f"It takes {round(time_a - time_b, 2)} seconds")

In [ ]:
print(f"""
          The number of drones: {num_drones}
          The number of simulation steps: {10_000}""")

# Running the optimization problem

In [ ]:
 from scipy.optimize import differential_evolution, dual_annealing, shgo, basinhopping

### Differential Evolution

In [ ]:
import time

time_b = time.time() # Initializing the timmer

# Create the boundaries list
bound_list = [(0, len(order_df))] * 15


# Defining the cost function simulation arguments
arguments = (products_df,  # Product DataFrame 
             warehouse_df, # Warehouse DataFrame
             order_df,     # Orders DataFrame
             num_drones,   # Number of drones 
             15,           # Number of deliveries per drone
             10_000)        # Number of simulation turns/steps


# Using the optimization algorithm
summary = differential_evolution(algorithm_cost,
                                 bound_list,
                                 args=arguments,
                                 maxiter=500,
                                 tol=0.05,
                                 recombination=0.7,
                                 #popsize=35,
                                 #updating='deferred',
                                 #workers=-1,
                                 disp=True
                                )

time_a = time.time() # Computing the time after

In [ ]:
summary

In [ ]:
print(f"Time to converge {time_a - time_b} seconds.")

In [ ]:
# Creating the drone orders
drone_orders = list(set([int(i) for i in summary.x]))

# Defining the drone movement
drone_orders_df = order_df.iloc[drone_orders]

# Getting the coordinates
x_coordinates = drone_orders_df["coor_x"].to_list()
y_coordinates = drone_orders_df["coor_y"].to_list()

# Plotting the drone deliveries
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=order_df["coor_x"],
    y=order_df["coor_y"],
    name="Orders",
    mode="markers",
    marker_symbol="circle",
    marker_size=5, 
))

fig.add_trace(go.Scatter(
    x=warehouse_df["col"],
    y=warehouse_df["row"],
    name="Warehouses",
    mode="markers",
    marker_symbol="hexagram",
    marker_size=20, 
))

fig.add_trace(go.Scatter(
    x=[0] + x_coordinates, 
    y=[0] + y_coordinates, 
    name="Deliveries", 
    mode="lines+markers", 
    marker_size=10
))

fig.add_trace(go.Scatter(
    x=[x_coordinates[0]],
    y=[y_coordinates[0]],
    name="First Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[x_coordinates[-1]],
    y=[y_coordinates[-1]],
    name="Last Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    name="Starting position",
    mode="markers",
    marker_symbol="star-square",
    marker_size=15, 
))
    
fig.show()

### Simmulated Annelaing

In [ ]:
time_b = time.time() # Initializing the timmer

# Create the boundaries list
bound_list = [(0, len(order_df))] * 15


# Defining the cost function simulation arguments
arguments = (products_df,  # Product DataFrame 
             warehouse_df, # Warehouse DataFrame
             order_df,     # Orders DataFrame
             num_drones,   # Number of drones 
             15,           # Number of deliveries per drone
             10_000)       # Number of simulation turns/steps


# Using the optimization algorithm
summary = dual_annealing(algorithm_cost,
                         bound_list,
                         args=arguments,
                         maxiter=500)

time_a = time.time() # Computing the time after

In [ ]:
summary

In [ ]:
print(f"Time to converge {time_a - time_b} seconds.")

In [ ]:
# Creating the drone orders
drone_orders = list(set([int(i) for i in summary.x]))

# Defining the drone movement
drone_orders_df = order_df.iloc[drone_orders]

# Getting the coordinates
x_coordinates = drone_orders_df["coor_x"].to_list()
y_coordinates = drone_orders_df["coor_y"].to_list()

# Plotting the drone deliveries
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=order_df["coor_x"],
    y=order_df["coor_y"],
    name="Orders",
    mode="markers",
    marker_symbol="circle",
    marker_size=5, 
))

fig.add_trace(go.Scatter(
    x=warehouse_df["col"],
    y=warehouse_df["row"],
    name="Warehouses",
    mode="markers",
    marker_symbol="hexagram",
    marker_size=20, 
))

fig.add_trace(go.Scatter(
    x=[0] + x_coordinates, 
    y=[0] + y_coordinates, 
    name="Deliveries", 
    mode="lines+markers", 
    marker_size=10
))

fig.add_trace(go.Scatter(
    x=[x_coordinates[0]],
    y=[y_coordinates[0]],
    name="First Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[x_coordinates[-1]],
    y=[y_coordinates[-1]],
    name="Last Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    name="Starting position",
    mode="markers",
    marker_symbol="star-square",
    marker_size=15, 
))
    
fig.show()

### Simplicial Homology Global Optimization

In [ ]:
time_b = time.time() # Initializing the timmer

# Create the boundaries list
bound_list = [(0, len(order_df)-1)] * 15


# Defining the cost function simulation arguments
arguments = (products_df,  # Product DataFrame 
             warehouse_df, # Warehouse DataFrame
             order_df,     # Orders DataFrame
             num_drones,   # Number of drones 
             15,           # Number of deliveries per drone
             1_000)       # Number of simulation turns/steps


# Using the optimization algorithm
summary = shgo(algorithm_cost,
              bound_list,
              args=arguments,
              )

time_a = time.time() # Computing the time after

In [ ]:
summary

In [ ]:
print(f"Time to converge {time_a - time_b} seconds.")

### Basin Hopping

In [ ]:
...

In [ ]:
summary

# Searching for each drone

In [ ]:
def one_per_time_cost(orders, drones_ready, product_df, warehouse_df, order_df, num_drones, num_orders_per_drone, num_turns):
    """
        Algorithm that based on the list of orders that should be delivered, will simulate all drones and return the 
        percentage of distance that this particular drone moved and the percentage of orders delivered for all the 
        drones.
    """
    
    # Getting the unique orders as integers
    order_ids = list(set([int(i) for i in orders]))
    
    # Creating the order list to be delivered
    order_list = []
    # Including the drones that are ready
    for drone_odf in drones_ready:
        for order in range(len(drone_odf)):
            # Get the specific order content
            num_items = drone_odf.iloc[order,-3]
            prod_codes = drone_odf.iloc[order, :num_items].to_list()
            coordinates = drone_odf.iloc[order, -2:].to_list()
            # Concat the order content
            order_list.append( (order, prod_codes, coordinates) )
    
    # Computing the order list for all drones
    for order in order_ids:
        # Get the specific order content
        num_items = order_df.iloc[order,-3]
        prod_codes = order_df.iloc[order, :num_items].to_list()
        coordinates = order_df.iloc[order, -2:].to_list()
        # Concat the order content
        order_list.append( (order, prod_codes, coordinates) )
    
    
    # Building the warehouse and product logger
    logger = build_logger(products_df, warehouse_df)
    
    # Creating the drones for each list created
    drones = create_drones(num_drones=num_drones, opd=num_orders_per_drone, orders=order_list, verbose=False)

    # Simulating the drones movements to deliveries
    sim_drones = simulate_drones(drones=drones, steps=num_turns, logger=logger)  
    
    
    # Compute the delivery cost
    distance = 0
    delivered_orders = 0
    for drone in sim_drones:
        distance += drone._distance / num_turns
        delivered_orders += drone._delivered_orders / len(orders)
    
    # Normalizing the distance (percentage)
    distance = 100 * distance / len(sim_drones)
    delivered_orders = 100 * delivered_orders
    
    
    # Return the cost for this simulation
    print(f"-> Testing cost at: {distance + (100 - delivered_orders)}")
    return distance + (100 - delivered_orders) # Mean proportion of distance and proportion of delivered orders

In [ ]:
drones = list(range(20))
dol = [] # Drone Order List

# Logging DataFrames
pdf = products_df.copy()
wdf = warehouse_df.copy()
odf = orders_df.reset_index().copy()

for drone_id in drones:

    # Create the boundaries list
    bound_list = [(0, len(odf))] * 35

    # Defining the cost function simulation arguments
    arguments = (dol.iloc[:,1:],   # The list of drones with already selected products
                 pdf,              # Product DataFrame 
                 wdf,              # Warehouse DataFrame
                 odf.iloc[:,1:],   # Orders DataFrame
                 drone_id + 1,     # Number of drones 
                 35,               # Number of deliveries per drone
                 10_000)           # Number of simulation turns/steps

    # Using the optimization algorithm
    summary = differential_evolution(one_per_time_cost,
                                     bound_list,
                                     args=arguments,
                                     # maxiter=6000,
                                     # popsize=35,
                                     updating='deferred',
                                     workers=-1,
                                     # disp=True
                                    )
    
    # Getting the drone orders from the optimizer
    selected_orders_ids = list(set([int(i) for i in summary.x]))
    # Get the orders already assigned to from the orders dataframe
    dol += [ odf[selected_orders_id] ]
    
    # Remove the orders assigned to other drones
    odf = odf.drop(selected_orders_id)
    
